In [1]:
import pandas as pd

In [3]:
matches = pd.read_csv("matches.csv", index_col = 0)
matches

,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
Season_End_Year,,,,,,,
1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
...,...,...,...,...,...,...,...
2023,38,2023-05-28,Everton,1,0,Bournemouth,H
2023,38,2023-05-28,Leicester City,2,1,West Ham,H
2023,38,2023-05-28,Aston Villa,2,1,Brighton,H


In [4]:
38 * 20 * 30

22800

In [5]:
matches["FTR"].value_counts()

FTR
H    5519
A    3410
D    3097
Name: count, dtype: int64

In [6]:
# checking the number of match weeks
matches["Wk"].value_counts()

Wk
1     313
30    313
23    313
24    313
28    313
25    313
26    313
27    313
29    313
33    313
21    313
31    313
32    313
34    313
35    313
36    313
37    313
38    313
2     313
22    313
20    313
10    313
3     313
4     313
5     313
6     313
7     313
8     313
19    313
9     313
11    313
12    313
13    313
14    313
15    313
16    313
17    313
18    313
40     33
39     33
41     33
42     33
Name: count, dtype: int64

In [7]:
matches.dtypes

Wk            int64
Date         object
Home         object
HomeGoals     int64
AwayGoals     int64
Away         object
FTR          object
dtype: object

In [8]:
matches["Date"] = pd.to_datetime(matches["Date"])

In [9]:
matches.dtypes

Wk                    int64
Date         datetime64[ns]
Home                 object
HomeGoals             int64
AwayGoals             int64
Away                 object
FTR                  object
dtype: object

In [11]:
matches["Home_adv_code"] = matches["Home"].astype("category").cat.codes
matches["Away_adv_code"] = matches["Away"].astype("category").cat.codes
matches["day_code"] = matches["Date"].dt.dayofweek
matches["target"] = (matches["FTR"] == "H").astype("int")

In [12]:
# machine learning model
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf  = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [15]:
# splitting the training data and test 
training = matches[matches["Date"] < '2023-01-01']

# testing data
testing  = matches[matches["Date"] > '2023-01-01']

In [17]:
predictors = ["Home_adv_code", "Away_adv_code", "day_code"]

#Fit model
rf.fit(training[predictors], training["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [18]:
newPredictors = rf.predict(testing[predictors])

from sklearn.metrics import accuracy_score
acc = accuracy_score(testing["target"], newPredictors)

# predicting the accuracy of the intial model 
acc

0.5841121495327103

In [19]:
combined = pd.DataFrame(dict(actual=testing["target"], prediction=newPredictors))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,75,32
1,57,50


In [20]:
from sklearn.metrics import precision_score
precision_score(testing["target"], newPredictors)

0.6097560975609756

In [21]:
group_matches = matches.groupby("Home")
group = group_matches.get_group("Liverpool")

group

,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,Home_adv_code,Away_adv_code,day_code,target
Season_End_Year,,,,,,,,,,,
1993,2,1992-08-19,Liverpool,2,1,Sheffield Utd,H,25,36,2,1
1993,3,1992-08-23,Liverpool,0,2,Arsenal,A,25,0,6,0
1993,6,1992-09-01,Liverpool,1,1,Southampton,D,25,38,1,0
1993,7,1992-09-05,Liverpool,2,1,Chelsea,H,25,14,5,1
1993,10,1992-09-26,Liverpool,2,3,Wimbledon,A,25,48,5,0
...,...,...,...,...,...,...,...,...,...,...,...
2023,32,2023-04-22,Liverpool,3,2,Nott'ham Forest,H,25,31,5,1
2023,34,2023-04-30,Liverpool,4,3,Tottenham,H,25,43,6,1
2023,28,2023-05-03,Liverpool,1,0,Fulham,H,25,19,2,1


In [23]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group


cols = ["HomeGoals", "AwayGoals"]
new_cols = [f"{c}_rolling" for c in cols]

new_cols

['HomeGoals_rolling', 'AwayGoals_rolling']

In [24]:
rolling_averages(group, cols, new_cols)

,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,Home_adv_code,Away_adv_code,day_code,target,HomeGoals_rolling,AwayGoals_rolling
Season_End_Year,,,,,,,,,,,,,
1993,7,1992-09-05,Liverpool,2,1,Chelsea,H,25,14,5,1,1.000000,1.333333
1993,10,1992-09-26,Liverpool,2,3,Wimbledon,A,25,48,5,0,1.000000,1.333333
1993,11,1992-10-03,Liverpool,1,0,Sheffield Weds,H,25,37,5,1,1.666667,1.666667
1993,13,1992-10-25,Liverpool,4,1,Norwich City,H,25,30,6,1,1.666667,1.333333
1993,15,1992-11-07,Liverpool,4,1,Middlesbrough,H,25,28,5,1,2.333333,1.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,32,2023-04-22,Liverpool,3,2,Nott'ham Forest,H,25,31,5,1,3.666667,0.666667
2023,34,2023-04-30,Liverpool,4,3,Tottenham,H,25,43,6,1,4.000000,1.333333
2023,28,2023-05-03,Liverpool,1,0,Fulham,H,25,19,2,1,3.000000,2.333333


In [26]:
matches_rolling = matches.groupby("Home").apply(lambda x: rolling_averages(x, cols, new_cols))

matches_rolling

C:\Users\mikoo\AppData\Local\Temp\ipykernel_2936\1601838008.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Home").apply(lambda x: rolling_averages(x, cols, new_cols))


Wk       Date     Home  HomeGoals  AwayGoals  \
Home    Season_End_Year                                                 
Arsenal 1993              8 1992-09-12  Arsenal          0          1   
        1993             10 1992-09-28  Arsenal          1          0   
        1993             11 1992-10-03  Arsenal          2          1   
        1993             13 1992-10-24  Arsenal          2          0   
        1993             15 1992-11-07  Arsenal          3          0   
...                      ..        ...      ...        ...        ...   
Wolves  2023             30 2023-04-08   Wolves          1          0   
        2023             31 2023-04-15   Wolves          2          0   
        2023             33 2023-04-25   Wolves          2          0   
        2023             35 2023-05-06   Wolves          1          0   
        2023             37 2023-05-20   Wolves          1          1   

                                    Away FTR  Home_adv_code  Away_adv_code  \
Home    Season_End_Year                                                      
Arsenal 1993                   Blackburn   A              0              4   
        1993             Manchester City   H              0             26   
        1993                     Chelsea   H              0             14   
        1993                     Everton   H              0             18   
        1993               Coventry City   H              0             15   
...                                  ...  ..            ...            ...   
Wolves  2023                     Chelsea   H             49             14   
        2023                   Brentford   H             49              9   
        2023              Crystal Palace   H             49             16   
        2023                 Aston Villa   H             49              1   
        2023                     Everton   D             49             18   

                         day_code  target  HomeGoals_rolling  \
Home    Season_End_Year                                        
Arsenal 1993                    5       0           2.000000   
        1993                    0       1           1.333333   
        1993                    5       1           1.000000   
        1993                    5       1           1.000000   
        1993                    5       1           1.666667   
...                           ...     ...                ...   
Wolves  2023                    5       1           1.000000   
        2023                    5       1           1.333333   
        2023                    1       1           1.666667   
        2023                    5       1           1.666667   
        2023                    5       0           1.666667   

                         AwayGoals_rolling  
Home    Season_End_Year                     
Arsenal 1993                      1.666667  
        1993                      0.666667  
        1993                      0.666667  
        1993                      0.666667  
        1993                      0.333333  
...                                    ...  
Wolves  2023                      1.666667  
        2023                      1.333333  
        2023                      1.333333  
        2023                      0.000000  
        2023                      0.000000  

[11876 rows x 13 columns]

In [27]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,Home_adv_code,Away_adv_code,day_code,target,HomeGoals_rolling,AwayGoals_rolling
0,8,1992-09-12,Arsenal,0,1,Blackburn,A,0,4,5,0,2.000000,1.666667
1,10,1992-09-28,Arsenal,1,0,Manchester City,H,0,26,0,1,1.333333,0.666667
2,11,1992-10-03,Arsenal,2,1,Chelsea,H,0,14,5,1,1.000000,0.666667
3,13,1992-10-24,Arsenal,2,0,Everton,H,0,18,5,1,1.000000,0.666667
4,15,1992-11-07,Arsenal,3,0,Coventry City,H,0,15,5,1,1.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11871,30,2023-04-08,Wolves,1,0,Chelsea,H,49,14,5,1,1.000000,1.666667
11872,31,2023-04-15,Wolves,2,0,Brentford,H,49,9,5,1,1.333333,1.333333
11873,33,2023-04-25,Wolves,2,0,Crystal Palace,H,49,16,1,1,1.666667,1.333333
11874,35,2023-05-06,Wolves,1,0,Aston Villa,H,49,1,5,1,1.666667,0.000000


In [29]:
def make_predictions(data, predictors):
    training = data[data["Date"] < '2023-01-01']
    testing  = data[data["Date"] > '2023-01-01']
    rf.fit(training[predictors], training["target"])
    preds = rf.predict(testing[predictors])
    combined = pd.DataFrame(dict(actual=testing["target"], predicted=preds), index=testing.index)
    precision = precision_score(testing["target"], preds)
    return combined,precision

In [30]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)
precision

0.6025641025641025

In [31]:
combined.head()


,actual,predicted
580,0,1
581,1,1
582,0,1
583,0,0
584,1,0


In [32]:
import joblib

joblib.dump(rf, "YNWA_model.pkl")

['YNWA_model.pkl']